In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader
from torchvision import datasets
# from torch.cuda.amp import autocast, GradScaler
import json
from PIL import Image
from torchsummary import summary


# Add CBAM (Channel & Spatial Attention)
class CBAMLayer(nn.Module):
    def __init__(self, in_channels):
        super(CBAMLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
    
    def forward(self, x):
        attention = torch.sigmoid(self.conv(x))
        return x * attention


In [3]:
# Base EfficientNet architecture details
base_model = [
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    "b0": (0, 224, 0.2),
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

In [ ]:
# Device Setup (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

version = "b3" #efficientnet version

# Data Preprocessing and Augmentation
transform = transforms.Compose([
    transforms.Resize((phi_values[version][1], phi_values[version][1])),  # Resize images to fit the model's input size
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load datasets (assuming you have train and test data in 'Processed_Data/train' and 'Processed_Data/test')
train_dataset = datasets.ImageFolder(root="Processed_Data/train", transform=transform)
test_dataset = datasets.ImageFolder(root="Processed_Data/test", transform=transform)

# DataLoader Setup
batch_size = 32  # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Save class names for inference
with open("class_names.json", "w") as f:
    json.dump(train_dataset.classes, f)


# Load Pretrained EfficientNet Model
model = EfficientNet.from_pretrained('efficientnet-b3')


# Modify Final Layer for the Number of Classes in Your Dataset
num_ftrs = model._fc.in_features
print(len(train_dataset.classes))
model._fc = nn.Linear(num_ftrs, len(train_dataset.classes))  # Adjust the number of classes

model.cbam = CBAMLayer(in_channels=num_ftrs)
# Move model to GPU if available
model = model.to(device)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, factor=0.1, verbose=True)

# Early Stopping Parameters
patience = 2  # Stop if validation loss doesn't improve for 'patience' epochs
best_val_loss = float('inf')
epochs_no_improve = 0

# Mixed Precision Training (FP16)
scaler = torch.amp.GradScaler('cuda')

def validate_model(model, test_loader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            with torch.amp.autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = val_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Training Function with Early Stopping
def train_model(model, train_loader, num_epochs):

    global best_val_loss, epochs_no_improve

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i,(images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.amp.autocast('cuda'):  # FP16 forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

            # Backpropagation with scaler
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            if i % 10 == 0:  # Print more frequently
                    print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate_model(model, test_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        scheduler.step(val_loss)  # Adjust learning rate if validation loss plateaus

        # Check for Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'crop_disease_model.pth')  # Save the best model
            print("✅ Model saved (new best validation loss)")
        else:
            epochs_no_improve += 1
            print(f"⚠️ No improvement in validation loss for {epochs_no_improve} epoch(s)")

        if epochs_no_improve >= patience:
            print(f"🚀 Early stopping at epoch {epoch+1}. Best Validation Loss: {best_val_loss:.4f}")
            break


# Train the Model
# train_model(model, train_loader, num_epochs=3)

Loaded pretrained weights for efficientnet-b3
22


In [ ]:
# # Load class names from JSON file
# with open("class_names.json", "r") as file:
#     class_names = json.load(file)  # Should be a list of class names


# # Load saved modael weights
# model.load_state_dict(torch.load("crop_disease_model.pth", map_location=device))
# model.to(device)
# model.eval()
    
# # Predict on a sample image
# image_path = "Processed_Data/test/Tomato_healthy/healthy2_.jpg"  # Replace with your image path
# predicted_disease, confidence = predict_image(image_path, model, class_names)

# print(f"Predicted Disease: {predicted_disease} (Confidence: {confidence:.2f})")

In [ ]:
#Clearing cache

# import torch
# import gc

# torch.cuda.empty_cache()  # Clears unused memory from the cache
# torch.cuda.ipc_collect()
# gc.collect()  # Force garbage collection
# torch.cuda.empty_cache()

In [20]:
import torch
import json
from torchvision import transforms
from PIL import Image

# ==========================
# PREDICTION FUNCTION (CROP-SPECIFIC)
# ==========================
def predict(image_path, crop_name, model, device):
    """
    Predicts the disease of the given image, considering only diseases from the specified crop.
    
    :param image_path: Path to the image file
    :param crop_name: The crop to filter diseases from (e.g., "Cashew")
    :param model: The trained model
    :param device: The device (CPU/GPU) for computation
    :return: Predicted crop name, disease name, and confidence score
    """

    # Define image transformations (same as training)
    transform = transforms.Compose([
        transforms.Resize((300, 300)),  # Resize to EfficientNet-B3 input size
        transforms.ToTensor(),          # Convert image to tensor
        transforms.Normalize(           # Normalize using ImageNet stats
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Load and preprocess the image
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        raise ValueError(f"Error loading image: {e}")

    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    # Move model to device
    model = model.to(device)
    model.eval()

    # Load class names
    with open("class_names.json", "r") as f:
        class_names = json.load(f)

    # Filter disease classes for the given crop
    crop_classes = [cls for cls in class_names if cls.startswith(crop_name)]
    if not crop_classes:
        raise ValueError(f"No diseases found for crop: {crop_name}")

    # Create index mapping for this crop
    crop_indices = [class_names.index(cls) for cls in crop_classes]

    # Run inference
    with torch.no_grad():
        output = model(image)

    # Extract relevant disease classes
    crop_output = output[:, crop_indices]  # Select only disease indices related to crop
    predicted_idx = torch.argmax(crop_output, dim=1).item()

    # Map back to disease name
    predicted_disease = crop_classes[predicted_idx]
    confidence = torch.nn.functional.softmax(crop_output, dim=1)[0][predicted_idx].item()

    # Extract only the disease name (removing crop prefix)
    disease_name = predicted_disease.split("_", 1)[1]

    print(f"✅ Predicted Crop: {crop_name}")
    print(f"✅ Predicted Disease: {disease_name} (Confidence: {confidence:.2f})")

    return crop_name, disease_name, confidence

# ==========================
# SAMPLE PREDICTION
# ==========================

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model.load_state_dict(torch.load("crop_disease_model.pth", map_location=device))
model.to(device)
model.eval()

# Define crop and image path
crop_name = "Tomato"  # Example crop input
image_path = "Sample Predict/tomato_leafblight.jpg"

# Run prediction
predict(image_path, crop_name, model, device)


C:\Users\Admin\AppData\Local\Temp\ipykernel_20912\3009905370.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("crop_disease_model.pth", 

✅ Predicted Crop: Tomato
✅ Predicted Disease: leaf curl (Confidence: 0.74)


('Tomato', 'leaf curl', 0.7434619069099426)

In [ ]:

# Print Model Summary
summary(model, input_size=(3, 300, 300))  # Adjust input size based on EfficientNet-B3


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 301, 301]               0
Conv2dStaticSamePadding-2         [-1, 40, 150, 150]           1,080
       BatchNorm2d-3         [-1, 40, 150, 150]              80
MemoryEfficientSwish-4         [-1, 40, 150, 150]               0
         ZeroPad2d-5         [-1, 40, 152, 152]               0
Conv2dStaticSamePadding-6         [-1, 40, 150, 150]             360
       BatchNorm2d-7         [-1, 40, 150, 150]              80
MemoryEfficientSwish-8         [-1, 40, 150, 150]               0
          Identity-9             [-1, 40, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 10, 1, 1]             410
MemoryEfficientSwish-11             [-1, 10, 1, 1]               0
         Identity-12             [-1, 10, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 40, 1, 1]             440
         I